In [ ]:
knitr::opts_chunk$set(cache = F)



In [ ]:
install.packages('tinytex')
tinytex::install_tinytex()


 
<font size= "5">**INTRODUCTION**</font>

In the traditional statistical sense, logistic regression is a way to estimate the parameters of a logistic model, where logistic models are commonly used to predict a dichotomous outcome by the linear combination of independent variables. Feature selection method is an automatic process that selects a subset of the independent variables(features) from the existing set of features to create a model. The premise of this method is the removal of features that are redundant and irrelevant without losing too much information. In some cases, these methods can provide models that are easier to interpret, can handle model complexity, and are less computationally intensive. 

For this presentation, we will be utilizing the ["Pima Indians Diabetes"](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database) data set to demonstrate the application of feature selection methods for logistic models. Using these methods, our goal is to create a model that can accurately predict our dichotomous outcome(diagnosis of Type 2 diabetes) from a selection of provided health information of patients. In addition, a demonstration of model interpretation for logistic regression will be provided to evaluate the effects of the provided health information of patients to the diagnosis of Type 2 diabetes.

It is important to note that the goal of this presentation is the application of logistic modeling with R. Therefore, many statistical concepts that are used in this presentation will be only be introduced, but links with additional information will be provided throughout the vignette.

### I. Data Cleaning


In [ ]:
diabetes <- read.csv("/Users/dancho/Downloads/diabetes.csv")



Summary of the diabetes data set:


In [ ]:
summary(diabetes)



Let's filter out the missing values that are denoted as 0.



In [ ]:
library(dplyr)



In [ ]:
diabetes <- filter(diabetes, Glucose > 0, BloodPressure > 0, SkinThickness > 0, Insulin > 0, BMI > 0)
head(diabetes)


In this dataset, there are eight candidates for our explanatory/predictor variables which include: Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, and Age. Our dichotomous response variable is Outcome, which has the values 0 or 1 to denote whether the patient was diagnosed with Type 2 diabetes or not.

### II. Logistic Regression

To fit a logistic regression model in R, we can use the glm function and family = binomial. Instead of the least squares method used in linear regression, this function for logistic regression uses [maximum likelihood estimation and log likelihood function](https://arunaddagatla.medium.com/maximum-likelihood-estimation-in-logistic-regression-f86ff1627b67) to estimate the coefficients, and find the best fitting line.


In [ ]:
model_test <- glm(Outcome ~ ., data = diabetes)
summary(model_test)


Similar to the lm function for linear regression, the summary displays our estimated coefficients and associated p-values.

In logistic regression, our "Y" is represented as log odds, which is equal to the linear combination of independent variables.

$$log(\frac{p}{1-p})=\beta_0+\beta_1X_1+\beta_2X_2...$$

[Log odds](https://stats.stackexchange.com/questions/130246/odds-ratio-in-logistic-regression-with-multiple-predictors) is denoted on the left, where [odds](https://towardsdatascience.com/probability-vs-odds-f47fbc6789f4) is the probability of success over the probability of failure. Each estimated coefficient is the expected change in log odds of our response variable, while holding our other variables as a constant. Finding the expected change in log odds would not be intuitively understood, so we have to convert the log odds to odds by exponentiating both sides.

First, when we exponentiate the linear combination of variables, it becomes a multiplicative expression.
$$\frac{p}{1-p} = e^{\beta_{0}+\beta_{1}X_{1}+\beta_{2}X_{2}}...$$
$$\frac{p}{1-p} = e^{\beta_{0}}e^{\beta_{1}X_{1}}e^{\beta_{2}X_{2}}...$$
Next, let's hold every other variable besides X1 as a constant.
$$\frac{p}{1-p} = e^{\beta_1X_1}C$$
Now when we want to find the expected change in the odds of our response variable for one unit increase(X), we are calculating the odds ratio between the new odds from the old odds.
$$\frac{e^{\beta_1(2)}C}{e^{\beta_1(1)}C}=e^{\beta_1(1)}$$
This was important to note since we will use this for our model interpretation at the end of this vignette.

### III. Correlation Matrix and Variance Inflation Factor

Before we start our feature selection, it is important to measure linear independence, and check for multi-collinearity. It is important to check for multi-collinearity because when our independent variables are highly correlated, the results of our model will very unstable to use for the purpose of prediction and statistical inference. For statistical inference, multi-collinearity leads to unstable coefficient estimates and p-values, making it difficult to properly interpret the model. For prediction, multi-collinearity can also lead to overfitting, where the model will perform inaccurately to out-of-sample data.


In [ ]:
library(ggcorrplot)
library(rcompanion)
library(car)


In [ ]:
diabetes %>%
cor(use = "all.obs") %>%
ggcorrplot(show.diag = F, type = "lower", lab = TRUE, lab_size=2)

data.frame(vif(model_test))


The [Pearson Correlation Matrix](https://stats.stackexchange.com/questions/113252/correlation-using-logistic-regression-and-pearson) shows us there is correlation among the independent variables which might suggest multi-collinearity. We will check the [variance inflation factor](https://medium.com/@analyttica/what-is-the-variance-inflation-factor-vif-d1dc12bb9cf5) to quantify the collinearity among our independent variables. The variation inflation factor scores suggest that the collinearity among our independent variables is relatively low, where a VIF above 5 indicates that the variables are highly correlated.

### IV. Model Building with Feature Selection Methods

We will first partition our data into training and testing data sets. This is important because we want to train our model to estimate the parameters and evaluate how well it will perform on new data.  To simulate this, we are using 80 percent of the dataset to train because we want to train our model with as much data as possible while leaving a portion of the dataset for testing. We do not want to reuse the same data for both training and testing because we want to know how well our model will categorize new data, or it will simply "memorize" the training data.


In [ ]:
set.seed(123) #reproducibility
#80% into train and 20% into test
samp <- sample(1:nrow(diabetes), 313)
train <- diabetes[samp, ]
test <- diabetes[-samp, ]


In [ ]:
library(bestglm)
library(caret)
library(InformationValue)
library(pROC)
library(glmnet)


There are many different types of feature selection methods, but we will only be demonstrating the best subset selection and LASSO feature selection methods.

Our first feature selection method we will use is [best subset selection](https://www.statology.org/best-subset-selection/). This procedure tests all the linear combinations of features by a specificied criterion: Adjusted R-squared, [Akaike information criterion](https://towardsdatascience.com/introduction-to-aic-akaike-information-criterion-9c9ba1c96ced), Bayesian information criterion, and Mallows's Cp. These criterions for model selection is a way to measure the goodness of fit between a collection of models. The criterion we will use for our best subset selection is the AIC, where one of the benefits of using AIC is that it includes a penalty term that can balance in-sample predictive performance with model complexity. The model with the lowest AIC, indicates the "best" model from our collection of models.


In [ ]:
best.logit_AIC <- bestglm(train, IC = "AIC", family = binomial, method = "exhaustive")
best.logit_AIC$Subsets


The model with the lowest(best) AIC score is shown as #4, where it selected the features: Glucose, BMI, DiabetesPedigreeFunction, and Age. We can see that it did not select the features that were correlated with the selected features. In addition, we can observe the penalty term in play as the AIC score increases with more than four features.

**Prediction with our best subset model.**

To see the performance of our trained model to our testing data, we will use the predict function with type = response, to output probabilities between zero and one. We will then use the confusionMatrix function to create a [confusion matrix](https://towardsdatascience.com/understanding-confusion-matrix-a9ad42dcfd62) to display how many of the predictions were properly classified by a specific threshold. The default threshold is 0.5 which makes sense intuitively because it is the middle point of the interval between zero and one. For example, if it predicted a value of 0.9, and the response variable in the testing set was 1, then it was properly classified since it's greater than 0.5. However, if we predicted values of 0.49 or 0.51, would it make sense to explicitly classify it? This can definitely create space for confusion.

This is where we can utilize the Receiver Operating Characteristics(ROC) curve and the Area Under Curve(AUC) to evaluate the classification performance. The ROC curve has two axes, Sensitivity and 1 - Specificity. Sensitivity is our true positive rate or the proportion of positives that were correctly predicted as positive, and 1 - Specificity is our false positive rate or the proportion of negatives that were wrongly predicted as positive. The benefit of using ROC curve is that is tests the model at all possible thresholds between 0 and 1, and be visualized. The AUC is our measure of performance at all thresholds as it computes the area under the the ROC curve, where a value of one indicates that the predictions were 100% correct, and has the shape of a right triangle that covers all the space under it.


In [ ]:
#train with selected model
best_train <- glm(Outcome ~ Glucose + BMI + DiabetesPedigreeFunction + Age, family = binomial, data = train)
#predict probability of response on a normalized scale
best_pred <- predict(best_train, test, type = "response")
#confusion matrix to display our true/false positives and negatives
confusionMatrix(test$Outcome, best_pred)
#Sensitivity displays our true positive rate
sensitivity(test$Outcome, best_pred)
#Specificity displays our true negative rate
specificity(test$Outcome, best_pred)
#ROC to measure TPR/FPR at all possible thresholds
ROC_best <- roc(test$Outcome, best_pred)
auc(ROC_best)


The [Area under the ROC curve](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) is 89.13% which indicates that our model performed pretty well.

The next feature selection method we will use is [LASSO](https://www.mygreatlearning.com/blog/understanding-of-lasso-regression/#:~:text=Lasso%20regression%20is%20a%20regularization,central%20point%20as%20the%20mean.),or least absolute shrinkage and selection operator. Similar to using AIC for best subset selection, this method performs regularization by adding a penalty term lambda, so that the model can generalize the data, instead of over-fitting. This type of regularization can deal with complexity and reduce over-fitting by increasing the value of our penalty term lambda, which shrinks the coefficients, and in some cases, eliminates features by outputting coefficients of zero. To perform the LASSO method for our logistic model, we have to use the glmnet function, alpha = 1, and family = binomial. 


In [ ]:
#Set our training data in matrix
x_train <- data.matrix(train[, c("Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
                                 "BMI", "DiabetesPedigreeFunction", "Age")])
#Set our testing data in matrix
x_test <- data.matrix(test[, c("Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
                               "BMI", "DiabetesPedigreeFunction", "Age")])

#Fit our lasso model with training data with standardization
lasso_train <- glmnet(x_train, y = as.factor(train$Outcome), alpha = 1, family = 'binomial')
#Display shrinkage of coefficients by lambda value
plot(lasso_train, xvar = 'lambda')


We can see here that our coefficients are shrinking towards zero as our lambda increases in value. We can continue to increase the value of lambda but our coefficients will continue to decrease to zero, so we need to find a way to select the optimal lambda value.

We can achieve this by finding the lambda value that minimizes the mean squared error. The cv.glmnet function uses k-fold cross validation to identify the "best" lambda value. K-fold cross validation is a method that divides our training set into k folds(default is 10), of equal size. It trains the fitted model on k-1 folds and calculates the test MSE on the fold that was left out, repeats this process for each fold, and the average of the k fold test MSE's is the overall test MSE. In addition, this process is applied for each lambda value to determine the lambda that minimizes the MSE.


In [ ]:
#k-fold cross-validation to find the best lambda
cv_lasso <- cv.glmnet(x_train, y = train$Outcome, alpha = 1)
#Best lambda value that minimizes mean squared error
best_lambda <- cv_lasso$lambda.min
#Display mean squared error by lambda value
plot(cv_lasso)


In [ ]:
lasso_best <- glmnet(x_train, y = as.factor(train$Outcome), alpha = 1, family = 'binomial', lambda = best_lambda)
#Display shrunk and eliminated coefficients
coef(lasso_best)


We can see here that the LASSO method eliminated two features where their coefficients are denoted as a period. In comparison to the coefficient estimates of the best subset selection model, these coefficient estimates are much smaller, and this model has two more features.

**Prediction with our LASSO model.**


In [ ]:
#Prediction probability for each observation with best lambda
lasso_pred <- predict(lasso_best, newx = x_test, type = "response")
#Find optimal probability cutoff score for our binary output
optimal_lasso <- optimalCutoff(test$Outcome, lasso_pred)[1]
#Confusion matrix to show false positives and false negatives
confusionMatrix(test$Outcome, lasso_pred)
#Sensitivity displays our true positive rate
sensitivity(test$Outcome, lasso_pred)
#Specificity displays our true negative rate
specificity(test$Outcome, lasso_pred)
#Percentage of total wrong classifications of model
misClassError(test$Outcome, lasso_pred, threshold = optimal_lasso)
lasso_pred2 <- as.vector(lasso_pred)
#ROC to measure TPR/FPR at all possible thresholds
ROC_lasso <- roc(test$Outcome, lasso_pred2)
auc(ROC_lasso)
par(pty = "s")
plot(ROC_lasso, col = "blue", main = "ROC For LASSO(BLUE) vs Best Subset AIC(ORANGE)", legacy.axes = T)
lines(ROC_best, col = "orange")


Our area under the ROC curve is 88.43% which is less than the best subset model. Plotting the two ROC curves, we can see that it agrees with the AUROC scores as the best subset curve is closer to a "right angle" shape. However, the difference between the two models is marginal due to the small size of the data set, number of features, and its non-complexity.  

In some cases, using the Area under the ROC Curve can have limitations depending on the context of the objective, where sensitivity will be weighted heavily to reduce the number of false negatives.

A limitation of using LASSO is that it does not provide us unbiased coefficient estimates and p-values to determine statistical significance, so we cannot properly interpret these estimated coefficients. More on that [here.](https://stats.stackexchange.com/questions/224796/why-are-confidence-intervals-and-p-values-not-reported-as-default-for-penalized)

### V. Model Interpretation for Logistic Regression

Let's evaluate the effects of the provided health information of patients to the diagnosis of  Type 2 diabetes with our best subset model.


In [ ]:
library(oddsratio)
best_model <- glm(Outcome ~ Glucose + BMI + DiabetesPedigreeFunction + Age, family = binomial, data = diabetes)
summary(best_model)


The p-values show that our predictors are all statistically significant.

We can use the or_glm function to find the expected change in the odds of our response variable for every one change of a predictor, while holding our other predictors as a constant.


In [ ]:
odds_ratio <- or_glm(data = diabetes, model = best_model,
                     incr = list(Glucose = 1, BMI = 1, DiabetesPedigreeFunction = 1, Age = 1)) #one unit increase
odds_ratio <- odds_ratio[,1:2]
odds_ratio$odds_percent_change <- (odds_ratio$oddsratio - 1) * 100 #percent change
odds_ratio


**Interpretation of p-values and coefficients**

For every one year increase in Age, we can expect a 5.4% increase in the odds of being diagnosed with diabetes while holding our other predictors as a constant. Age has a statistically significant relationship with the diagnosis of Type 2 diabetes. Naturally, we are more prone to diseases as we get older.

For every one increase in Glucose, we can expect a 3.7% increase in the odds of being diagnosed with diabetes while holding our other predictors as a constant. Glucose has a statistically significant relationship with the diagnosis of Type 2 diabetes. Type 2 diabetes is a disease where your body has difficulty lowering glucose levels.

For every one increase in BMI, we can expect a 7.7% increase in the odds of being diagnosed with diabetes while holding our other predictors as a constant. BMI has a statistically significant relationship with the diagnosis of Type 2 diabetes. Being overweight can lead to resistance to insulin which uses glucose for energy.

DiabetesPedigreeFunction is a "function that scores the likelihood of diabetes based on family history", that have values between 0 and 2.5, with a median value of 0.3725. This is a scaling issue that can be fixed with standardization but we would face another challenge of having an expected change in odds for every one standard deviation as opposed to a one unit increase.

For every one increase in DiabetesPedigreeFunction, we can expect a 196.6% increase in the odds of being diagnosed with diabetes while holding our other predictors as a constant. DiabetesPedigreeFunction has a statistically significant relationship with the diagnosis of Type 2 diabetes. Genetic factors can affect the development of type 2 diabetes.

This is a great article that explains the importance of [interpreting machine learning models](https://towardsdatascience.com/interpretability-in-machine-learning-70c30694a05f).

******

**Links to important packages**  
[bestglm](https://www.rdocumentation.org/packages/bestglm/versions/0.37.3/topics/bestglm)
[caret](https://www.rdocumentation.org/packages/caret/versions/6.0-93)
[pROC](https://www.rdocumentation.org/packages/pROC/versions/1.18.0)
[glmnet](https://www.rdocumentation.org/packages/glmnet/versions/4.1-4)
[oddsratio](https://www.rdocumentation.org/packages/oddsratio/versions/2.0.1)
[car](https://www.rdocumentation.org/packages/car/versions/3.1-0) 
